# Split AHN raw data into subtiles and generate matching DTM

Raw AHN pointclouds are rather large to process (5x6.25 km, up to 10GB), therefore we first further subdivide them into more manageable chunks while also making sure that the whole city area is covered.

Subtiles can be of fixed square dimensions (e.g. 1x1 km).

In [ ]:
import numpy as np
import pathlib
import laspy
import shapely.geometry as sg
import geopandas as gpd
from tqdm.notebook import tqdm

from upcp.utils import clip_utils

import set_path
from gvl.helper_functions import (
    roundup,
    rounddown,
    box_to_name,
    get_tilecode_from_filename,
    process_ahn_las_tile,
)

Note: the folder structure in the following cell serves for illustation purposes only.

In [ ]:
BASE_FOLDER = "../datasets/AHN4"

ahn_raw_folder = f"{BASE_FOLDER}/Bronbestanden/"
ahn_subtile_folder = f"{BASE_FOLDER}/AMS_subtiles_1000/"

# Dimension of each tile in meters (both sides have the same length)
tile_size = 1000
# Buffer around each tile (in meters) to have overlap.
tile_buffer = 5

pathlib.Path(ahn_subtile_folder).mkdir(parents=True, exist_ok=True)

## Data preparation

### Read stadsdelen shapes

In [ ]:
# Download "stadsdelen" JSON data
import urllib.request, json

with urllib.request.urlopen(
    "https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=INDELING_STADSDEEL&THEMA=gebiedsindeling"
) as url:
    data = json.loads(url.read().decode())

# Convert to GeoDataFrame
ams_gdf = gpd.GeoDataFrame.from_features(data["features"], crs="WGS84")
ams_gdf.to_crs(crs="epsg:28992", inplace=True)

In [ ]:
# Get bounds of city area
bounds = ams_gdf.unary_union.bounds

# Round bounds up / down to nearest multiple of box_size
ams_bounds = (
    rounddown(bounds[0], tile_size),
    rounddown(bounds[1], tile_size),
    roundup(bounds[2], tile_size),
    roundup(bounds[3], tile_size),
)

## Generate subtiles for the target area

In [ ]:
# Create grid of square subtiles with dimensions box_size spanning the city area
xs = np.arange(ams_bounds[0], ams_bounds[2], tile_size)
ys = np.arange(ams_bounds[1], ams_bounds[3], tile_size)

geoms = [sg.box(x, y, x + tile_size, y + tile_size) for x in xs for y in ys]
names = [box_to_name(box, tile_size) for box in geoms]

ams_subtiles_gdf = gpd.GeoDataFrame({"name": names, "geometry": geoms})

# target_shape = so.unary_union([all_tiles_merged.unary_union, ams_gdf.unary_union])
target_shape = ams_gdf.unary_union

ams_subtiles_gdf = ams_subtiles_gdf[ams_subtiles_gdf.intersects(target_shape)]

In [ ]:
# List all LAZ files, one example tile from geotiles.nl: https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/01_LAZ/C_25EZ1.LAZ
ahn_raw_files = list(pathlib.Path(ahn_raw_folder).glob("*.LAZ"))

# Compute bounds for each file
bounds = []
for f in ahn_raw_files:
    with laspy.open(f) as las:
        [x_min, y_min] = las.header.mins[0:2]
        [x_max, y_max] = las.header.maxs[0:2]
        bounds.append([x_min, y_min, x_max, y_max])

# Add all shapes to GeoDataFrame
ahn_raw_gdf = gpd.GeoDataFrame(
    {
        "filename": [f.name for f in ahn_raw_files],
        "geometry": [sg.box(*b) for b in bounds],
    }
)

ahn_raw_gdf = ahn_raw_gdf[ahn_raw_gdf.intersects(ams_subtiles_gdf.unary_union)]

In [ ]:
# Check existing files (ignore this cell to re-run for all tiles)
existing_subtiles = list(pathlib.Path(ahn_subtile_folder).glob("*.laz"))
existing_subtiles = {subtile.name[-11:-4] for subtile in existing_subtiles}

ams_subtiles_gdf["exists"] = ams_subtiles_gdf.name.isin(existing_subtiles)
ams_subtiles_gdf = ams_subtiles_gdf[~ams_subtiles_gdf.exists]

if len(ams_subtiles_gdf) == 0:
    print("All tiles already created, nothing else to do.")
else:
    ahn_raw_gdf["needed"] = ahn_raw_gdf.apply(
        lambda row: row["geometry"].intersects(ams_subtiles_gdf.unary_union), axis=1
    )
    ahn_raw_gdf = ahn_raw_gdf[ahn_raw_gdf.needed]

### Plotting (optional)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(5, 4))

ahn_raw_gdf.plot(ax=ax, color="grey", alpha=0.25)
ams_subtiles_gdf.plot(ax=ax, color="grey", edgecolor="black", alpha=0.5)
ams_gdf.plot(ax=ax, color="lightblue", alpha=0.5)

## Perform splitting

In [ ]:
# We use chunked reading & writing to prevent memory issues.
# Set number of points to be read for each chunk.
points_per_iter = int(1e6)

In [ ]:
# Add buffer and compute bounds.
ams_subtiles_gdf["geometry"] = ams_subtiles_gdf.buffer(tile_buffer)
ams_subtiles_gdf["bounds"] = ams_subtiles_gdf.apply(
    lambda row: row["geometry"].bounds, axis=1
)

In [ ]:
# Reset index for easier looping in code below.
ams_subtiles_gdf.reset_index(inplace=True, drop=True)

In [ ]:
# Compute total number of points to read for progress bar.
ahn_total_points = 0
for f in list(ahn_raw_gdf["filename"]):
    file_path = pathlib.Path(ahn_raw_folder) / f
    with laspy.open(file_path) as las:
        ahn_total_points += las.header.point_count

In [ ]:
from typing import List
from typing import Optional

pbar = tqdm(
    total=ahn_total_points,
    unit=" points",
    unit_scale=True,
    unit_divisor=1000,
    smoothing=0,
)

writers: List[Optional[laspy.LasWriter]] = [None] * len(ams_subtiles_gdf)

for _, row in ahn_raw_gdf.iterrows():
    # Loop over AHN raw source tiles
    ahn_file = row["filename"]
    ahn_shape = row["geometry"]
    pbar.set_postfix_str(ahn_file)
    file_path = pathlib.Path(ahn_raw_folder) / ahn_file
    with laspy.open(file_path) as file:
        try:
            for points in file.chunk_iterator(points_per_iter):
                # For performance we need to use copy
                # so that the underlying arrays are contiguous
                points_xy = np.vstack((points.x.copy(), points.y.copy())).T

                for i, row in ams_subtiles_gdf.iterrows():
                    # Loop over target subtiles.

                    # Check if the subtile intersects the source tile, if not skip.
                    if not sg.box(*row["bounds"]).intersects(ahn_shape):
                        continue

                    # Clip the points from this chunck that fall within the subtile.
                    mask = clip_utils.rectangle_clip(points_xy, row["bounds"])

                    # Write the points to the corresponding subtile.
                    if np.any(mask):
                        if writers[i] is None:
                            output_path = (
                                pathlib.Path(ahn_subtile_folder)
                                / f"ahn4_{row['name']}.laz"
                            )
                            writers[i] = laspy.open(
                                output_path, mode="w", header=file.header
                            )
                        sub_points = points[mask]
                        writers[i].write_points(sub_points)

                pbar.update(len(points))
        finally:
            pass
pbar.close()

for writer in writers:
    if writer is not None:
        writer.close()

## Generate matching DTM as .npz files

In [ ]:
# Generate .npz data for all tiles
ahn_npz_folder = f"{BASE_FOLDER}/npz_subtiles_1000/"

# Set the resolution for the DTM
dtm_resolution = 0.5

files = list(pathlib.Path(ahn_subtile_folder).glob("ahn*.laz"))
pathlib.Path(ahn_npz_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
# Check existing files (ignore this cell to re-run for all tiles)
existing_files = list(pathlib.Path(ahn_npz_folder).glob("*.npz"))
existing_codes = {get_tilecode_from_filename(file.name) for file in existing_files}

files = [
    file
    for file in files
    if get_tilecode_from_filename(file.name) not in existing_codes
]

In [ ]:
# Generate .npz files for each subtile, containing the ground surface (maaiveld).
file_tqdm = tqdm(files, unit="file", smoothing=0)
for file in file_tqdm:
    process_ahn_las_tile(file, out_folder=ahn_npz_folder, resolution=dtm_resolution)

### Plotting (optional)

In [ ]:
# Optional: load one DTM tile and visualize the data.
data = np.load(f"{BASE_FOLDER}/npz_subtiles_1000/ahn_118_486.npz")

%matplotlib widget
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(9, 6))

ax = fig.add_subplot(111)
ax.set_title("colorMap")
plt.imshow(data["ground"])
ax.set_aspect("equal")

cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
cax.get_xaxis().set_visible(False)
cax.get_yaxis().set_visible(False)
cax.patch.set_alpha(0)
cax.set_frame_on(False)
plt.colorbar(ax=cax, orientation="vertical")
plt.show()